# Model przerabiający liczbę na rzymską cyfrę
- input: number in range(0,DATASE_SIZE)
- output: roman number

Examples: 
- input: 10, output: X
- input: 6, output: VI

Import modułów

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RNN, LSTM, RepeatVector
from tensorflow.python.keras import layers
from encoder import CharacterTable 
import numpy as np
import roman_numerals as cnv

## Model

Zdefiniowanie znaków rzymskich i cyfr, utworzenie modelu

In [2]:
# Zdefiniowanie parametrów dla modelu i danych
INPUT = 3
OUTPUT = 10
DATASET_SIZE=200

# object to encode roman numbers to one-hot 
romans = ' MDCLXVI'
rtable = CharacterTable(romans)

# object to encode arabic numbers to one-hot
chars = ' 0123456789'
dtable = CharacterTable(chars)

# zbudowanie modelu
model = Sequential()
model.add(Dense(INPUT, input_dim=1)) 
model.add(RepeatVector(OUTPUT)) #długość wyniku końcowego
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(Dense(len(romans),activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 6         
_________________________________________________________________
repeat_vector (RepeatVector) (None, 10, 3)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 10, 128)           67584     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 64)            49408     
_________________________________________________________________
dense_1 (Dense)              (None, 10, 8)             520       
Total params: 117,518
Trainable params: 117,518
Non-trainable params: 0
_________________________________________________________________


## Dataset creation

Utworzenie zbioru danych

In [3]:
samples = []
labels = []
seq_samples = []
seq_labels = []

import random

for i in range(DATASET_SIZE):
    samples.append(i + 1)
    seq_samples.append(i + 1)
    words = cnv.convert(i + 1)
    labels.append(list(words))
    seq_labels.append(list(words))

samples = np.array(samples)
labels = np.array(labels)

print("Sample (input):",samples[0])
print("Label",labels[0])


nlabels = np.zeros((DATASET_SIZE,OUTPUT,len(romans)))
for i in range(DATASET_SIZE):
    for j in range(OUTPUT):
        if j>=len(labels[i]): 
                nlabels[i][j][0]=1
                continue
        x = labels[i][j]
        index = romans.index(x)
        nlabels[i][j][index] = 1
print("Label encoded (output):\n",nlabels[123])
labels = nlabels
print(labels.shape)
print(samples.shape)

Sample (input): 1
Label ['I']
Label encoded (output):
 [[0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]]
(200, 10, 8)
(200,)


C:\Users\pawel\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


Rozdzielenie zbioru danych na dane treningowe i testowe

In [4]:
TRAINING_SIZE = .5
from sklearn.model_selection import train_test_split
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE,random_state=42)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

Training samples: 100  test samples 100


Przetwarzanie label na rzymski odpowiednik

In [5]:
def label2roman(label):
    s = ''
    for r in label:
        s+=romans[int(r)]
        #print(i,'->',s)
    return s.strip()    
    
def check_model(verbose=0,show_training=1):
    pred = model.predict(samples)
    res = pred.argmax(axis=2)
    correct = 0
    for i in range(len(pred)):
        if(not show_training and i in trainSamples): continue
        train=''
        if (i + 1) in trainSamples: train='[T]'
        txt = label2roman(res[i])
        txt_correct = cnv.convert(i + 1)
        ok=''
        if(txt==txt_correct): 
            correct+=1
            ok = "[ok]"
        if(verbose==1):
            print(i + 1,'->',txt, ok,train)
    if verbose==0:
        for i in range(5):        
            x = random.randrange(DATASET_SIZE)
            print(x,'->',label2roman(res[x]))    
    print('Correct',correct,'of',len(pred),' = ',(correct/len(pred)))
    return correct,len(pred),(correct/len(pred))
check_model(1)

1 -> MMMMMMMMMM  [T]
2 -> MMMMMMMMMM  [T]
3 -> MMMMMMMMMM  
4 -> MMMMMMMMMM  [T]
5 -> MMMMMMMMMM  [T]
6 -> MMMMMMMMMM  
7 -> MMMMMMMM  
8 ->   [T]
9 ->   [T]
10 ->   
11 ->   [T]
12 ->   
13 ->   
14 ->   [T]
15 ->   [T]
16 ->   
17 ->   
18 ->   [T]
19 ->   
20 ->   
21 ->   [T]
22 ->   [T]
23 ->   
24 ->   [T]
25 ->   
26 ->   [T]
27 ->   
28 ->   
29 ->   [T]
30 ->   
31 ->   
32 ->   
33 ->   [T]
34 ->   
35 ->   [T]
36 ->   
37 ->   
38 ->   [T]
39 ->   
40 ->   [T]
41 ->   [T]
42 ->   
43 ->   
44 ->   [T]
45 ->   [T]
46 ->   
47 ->   
48 ->   [T]
49 ->   [T]
50 ->   [T]
51 ->   [T]
52 ->   
53 ->   [T]
54 ->   [T]
55 ->   [T]
56 ->   
57 ->   
58 ->   [T]
59 ->   [T]
60 ->   [T]
61 ->   
62 ->   
63 ->   [T]
64 ->   [T]
65 ->   [T]
66 ->   
67 ->   
68 ->   
69 ->   
70 ->   
71 ->   [T]
72 ->   [T]
73 ->   [T]
74 ->   
75 ->   [T]
76 ->   
77 ->   
78 ->   
79 ->   
80 ->   
81 ->   [T]
82 ->   [T]
83 ->   
84 ->   [T]
85 ->   
86 ->   
87 ->   
88 ->   [T]
89 ->   [T]
90 ->   

(0, 200, 0.0)

In [6]:
num_epochs = 0
EPOCHS=400
BATCH_SIZE = int(len(trainSamples)/5)
print('Training with',len(trainSamples),'samples',EPOCHS,'epochs and batch_size=',BATCH_SIZE)
print("Epochs so far",num_epochs)
for x in range(10):
    H = model.fit(trainSamples, trainLabels, epochs=EPOCHS,verbose=0,batch_size=BATCH_SIZE, validation_data=(testSamples, testLabels))
    num_epochs += EPOCHS
    print()
    print("Epoch {} - loss ={:6.3f}, loss improvement ={:6.3f}".
          format(num_epochs,H.history['loss'][-1], H.history['loss'][0]-H.history['loss'][-1]))
    pred = model.predict(samples)
    res = pred.argmax(axis=2)
    c,l,p = check_model()
    print("accuracy={:6.3f}%".format(100*p))
print("Done")



Training with 100 samples 400 epochs and batch_size= 20
Epochs so far 0

Epoch 400 - loss = 0.383, loss improvement = 1.222
164 -> CLXII
140 -> CXXXI
106 -> CXI
134 -> CXXXI
11 -> XI
Correct 22 of 200  =  0.11
accuracy=11.000%

Epoch 800 - loss = 0.304, loss improvement = 0.075
24 -> XXVV
10 -> XI
42 -> XLI
159 -> CLVI
87 -> LXXXII
Correct 34 of 200  =  0.17
accuracy=17.000%

Epoch 1200 - loss = 0.220, loss improvement = 0.064
44 -> XLVV
83 -> LXXXIV
156 -> CLVI
107 -> CVI
137 -> CXXXI
Correct 45 of 200  =  0.225
accuracy=22.500%

Epoch 1600 - loss = 0.226, loss improvement =-0.004
39 -> XL
146 -> CXLVII
55 -> LV
48 -> LL X
181 -> CLXXXI
Correct 50 of 200  =  0.25
accuracy=25.000%

Epoch 2000 - loss = 0.192, loss improvement = 0.022
52 -> LI
190 -> CXCIII
178 -> CLXXXI
73 -> LXXIII
57 -> LVI
Correct 56 of 200  =  0.28
accuracy=28.000%

Epoch 2400 - loss = 0.154, loss improvement = 0.030
68 -> LXXI
130 -> CXXXI
193 -> CXCII
107 -> CVI
9 -> XX
Correct 59 of 200  =  0.295
accuracy=29.500%

Sprawdzenie modelu i jego zapis

In [7]:
check_model(1)
model.save('model_number2roman.h5')

1 -> I [ok] [T]
2 -> II [ok] [T]
3 -> II  
4 -> IV [ok] [T]
5 -> V [ok] [T]
6 -> V  
7 -> VIII  
8 -> VIII [ok] [T]
9 -> IX [ok] [T]
10 -> XI  
11 -> XI [ok] [T]
12 -> XI  
13 -> XIV  
14 -> XIV [ok] [T]
15 -> XV [ok] [T]
16 -> XV  
17 -> XVIII  
18 -> XVIII [ok] [T]
19 -> XVIII  
20 -> XXI  
21 -> XXI [ok] [T]
22 -> XXII [ok] [T]
23 -> XXIV  
24 -> XXIV [ok] [T]
25 -> XXVI  
26 -> XXVI [ok] [T]
27 -> XXVI  
28 -> XXIX  
29 -> XXIX [ok] [T]
30 -> XXIX  
31 -> XXIII  
32 -> XXXIII  
33 -> XXXIII [ok] [T]
34 -> XXXVII  
35 -> XXXV [ok] [T]
36 -> XXXV  
37 -> XXXVIII  
38 -> XXXVIII [ok] [T]
39 -> XX VII  
40 -> XL [ok] [T]
41 -> XLI [ok] [T]
42 -> XLI  
43 -> XLIV  
44 -> XLIV [ok] [T]
45 -> XLV [ok] [T]
46 -> XLVI [ok] 
47 -> XLVIII  
48 -> XLVIII [ok] [T]
49 -> XLIX [ok] [T]
50 -> L [ok] [T]
51 -> LI [ok] [T]
52 -> LII [ok] 
53 -> LIII [ok] [T]
54 -> LIV [ok] [T]
55 -> LV [ok] [T]
56 -> LV  
57 -> LVIII  
58 -> LVIII [ok] [T]
59 -> LIX [ok] [T]
60 -> LX [ok] [T]
61 -> LX  
62 -> LXIII 

Sprawdzenie działania dla własnej liczby

In [8]:
input=10
x = model.predict(np.array([input]))
v = np.argmax(x,axis=2)
print(label2roman(v.ravel()))

XI
